<a href="https://colab.research.google.com/github/JiseokSeo/Toy_Projects/blob/main/%EB%8D%B0%EC%9D%B4%EC%BD%982.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !mkdir data2
# !mkdir data2/train
# !mkdir data2/train/image
# !mkdir data2/train/answer
# !mkdir data2/test
# !mkdir data2/test/image

In [ ]:
# !unzip /content/drive/MyDrive/DACON/data2/dirty_mnist_2nd.zip -d /content/data2/train/image
# !unzip /content/drive/MyDrive/DACON/data2/test_dirty_mnist_2nd.zip -d /content/data2/test/image
# !cp /content/drive/MyDrive/DACON/data2/dirty_mnist_2nd_answer.csv /content/data2/train/answer
# !cp /content/drive/MyDrive/DACON/sample_submission.csv /content

### 문제정의
노이즈가 낀 img에서 특정 알파벳이 포함되어있는지를 찾는 멀티라벨 분류문제입니다.

img는 여러 알파벳과 노이즈(쓸모없는 단어 등)이 포함되어 있으며,
답안은 a-z와 대응되는 길이로 0, 1로 매핑된 csv입니다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from google.colab import output
import csv
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 디바이스 설정

In [ ]:
answer = pd.read_csv('/content/dirty_mnist_2nd_answer.csv')

In [ ]:
answer = answer.drop(columns='index')
answer

,a,b,c,d,e,f,g,h,i,j,...,q,r,s,t,u,v,w,x,y,z
0,1,1,0,1,0,1,0,0,0,0,...,0,1,1,0,1,0,0,1,1,1
1,1,0,0,1,0,1,0,1,0,1,...,0,1,0,1,0,0,0,0,1,1
2,0,0,0,0,0,0,0,0,1,1,...,0,0,1,1,1,0,1,1,1,0
3,0,0,1,0,0,0,1,1,0,0,...,1,0,1,1,0,1,1,0,1,0
4,0,1,0,1,0,1,0,1,1,0,...,0,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0,1,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,1,1,0
49996,0,1,0,1,0,1,1,1,0,1,...,0,1,1,1,0,1,0,0,0,1
49997,0,1,0,0,1,1,1,1,0,0,...,1,0,0,0,0,1,1,1,0,0
49998,0,1,1,1,0,0,1,1,0,1,...,0,1,1,1,0,0,0,1,0,0


In [ ]:
# img 경로와 답안으로 구성된 metadata csv를 만드려고 합니다 

import os

root = '/content/data2/train'
image_path = '/content/data2/train/image'
image_paths = [os.path.join(image_path, path) for path in os.listdir(image_path)]

In [ ]:
image_paths = sorted(image_paths)

In [ ]:
# img 경로
image_paths[:10]

In [ ]:
data = []
for i in range(len(image_paths)):
  a = image_paths[i]
  b = answer.iloc[i].tolist()
  data.append([a, b])

metadata = pd.DataFrame(columns=['path', 'target'], data=data)

In [ ]:
# 경로와 답안이 저장된 메타데이터 완성
metadata.head(3)

In [ ]:
metadata['target'][0]

In [ ]:
# 이미지 경로를 입력하면 정규화하여 np array로 반환하는 함수
def get_image(path):
  img = cv2.imread(path)
  img = (img / 255).astype('float')
  return img

In [ ]:
# 이미지 1개 시각화
plt.imshow(get_image(metadata['path'][0]))

In [ ]:
# numpy 이미지의 shape 확인
get_image(metadata['path'][0]).shape

# 확인 결과 (256, 256, 3)로 3채널로 구성되었다.
# => 1채널(그레이스케일)로 바꿔도 되지만, 3채널로 학습된 프리트레인 모델을 쓸 것이라 그냥 두겠다
# => (H, W, C)의 형태롤 (C, H, W)로 바꿔주어야겠다. (이미지 증강단계에서)

In [ ]:
# 랜덤하게 n개의 샘플을 뽑아 시각화하는 함수 
def show_sample_image(num_sample=10):
  samples = metadata.sample(num_sample).reset_index(drop=True)
  paths = samples['path'].tolist()

  f, ax = plt.subplots(1, num_sample,
                      figsize=(24, 12))
  ax = ax.flatten()

  for idx in range(num_sample):
    ax[idx].imshow(get_image(paths[idx]))
    ax[idx].axis("off")

    plt.tight_layout()
    plt.show

In [ ]:
show_sample_image(10)

### 이미지 증강 

In [ ]:
import albumentations
from albumentations.pytorch.transforms import ToTensorV2
from typing import Tuple, Sequence, Callable
from torch import nn, Tensor
from torchvision import transforms

In [ ]:
transform = albumentations.Compose([ToTensorV2()])

### 데이터셋 구축 

In [ ]:
class BuildDataset(Dataset):
  def __init__(self, meatadata, transform=None):
    self.metadata = metadata
    self.transform = transform

  def __len__(self):
    return len(self.metadata)
    
  def __getitem__(self, idx):
    path = self.metadata['path'][idx]
    img = get_image(path)
    img = self.transform(image=img)['image']
    
    target = self.metadata.loc[idx, 'target']
    target = torch.tensor(target)

    return img, target

### 모델링 

In [ ]:
from torchvision.models import resnet50

In [ ]:
class ResNet50(nn.Module):
  def __init__(self):
    super().__init__()
    self.resnet = resnet50(pretrained=True)
    self.classifier = nn.Linear(1000, 26)

  def forward(self, x):
    x = self.resnet(x)
    x = self.classifier(x)
    return x

In [ ]:
model = ResNet50()
model.to(device)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

ResNet50(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          

### 모델 훈련 

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, valid = train_test_split(metadata,
                                test_size=0.2,
                                shuffle=True)

In [ ]:
train_dataset = BuildDataset(train, transform)
valid_dataset = BuildDataset(valid, transform)

train_dataloader = DataLoader(train_dataset, batch_size=256)
valid_dataloader = DataLoader(valid_dataset, batch_size=128)

In [ ]:
epochs = 10
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MultiLabelSoftMarginLoss()

In [ ]:
def to_device(img, target):
  img = img.to(device)
  target = target.to(device)
  return img, target

In [27]:
for epoch in range(epochs):
  iterator = tqdm(train_dataloader)
  for img, target in iterator:
    #img, target = sample['image'], sample['label']
    optimizer.zero_grad()

    img, target = to_device(img.float(), target)

    preds = model(img)
    loss = criterion(preds, target)

    loss.backward()
    # for name, param in model.named_parameters():
    #     print(name, param.grad)
    optimizer.step()

    preds = preds > 0.5
    acc = (preds == target).float().mean()
    iterator.set_description(f'epoch:{epoch+1}, loss:{loss.item():.5f}, acc:{acc.item():.5f}')

epoch:10, loss:0.21453, acc:0.91827: 100%|██████████| 782/782 [12:45<00:00,  1.02it/s]


In [32]:
torch.save(model.state_dict(), '/content/drive/MyDrive/DACON/data2/model_saved.pth')

### 검증 

In [36]:
model = ResNet50()
model.load_state_dict(torch.load('/content/drive/MyDrive/DACON/data2/model_saved.pth'))
model.eval()
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MultiLabelSoftMarginLoss()

In [37]:
valid_acc_list = []

iterator = tqdm(train_dataloader)
for img, target in iterator:
  img, target = to_device(img.float(), target)
  preds = model(img)
  preds = preds > 0.5

  acc = (preds == target).float().mean()
  valid_acc_list.append(acc)

print(sum(valid_acc_list)/len(valid_acc_list))

100%|██████████| 782/782 [03:09<00:00,  4.13it/s]

tensor(0.8475, device='cuda:0')


In [41]:
sum(valid_acc_list) / len(valid_acc_list)

tensor(0.8475, device='cuda:0')

### 추론

In [74]:
# 테스트 메타데이터 생성
test_image_path = '/content/data2/test/image'
test_image_paths = [os.path.join(image_path, path) for path in os.listdir(image_path)]
test_image_paths = sorted(test_image_paths)

test_data = []
for i in range(len(test_image_paths)):
  a = test_image_paths[i]
  b = [0] * 26
  test_data.append([a, b])

test_metadata = pd.DataFrame(columns=['path', 'target'], data=test_data)
test_metadata.head(3)

,path,target
0,/content/data2/train/image/00000.png,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,/content/data2/train/image/00001.png,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,/content/data2/train/image/00002.png,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [78]:
test_dataset = BuildDataset(test_metadata, transform)
test_dataloader = DataLoader(test_dataset,
                             batch_size=1)

In [79]:
submit = pd.read_csv('/content/sample_submission.csv')

In [80]:
submit

,index,a,b,c,d,e,f,g,h,i,...,q,r,s,t,u,v,w,x,y,z
0,50000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,50001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,50002,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,50003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50004,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,54995,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4996,54996,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4997,54997,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4998,54998,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
batch_size = 1
model = ResNet50()
model.load_state_dict(torch.load('/content/drive/MyDrive/DACON/data2/model_saved.pth'))
model.eval()
model.to(device)


for i, (img, target) in enumerate(tqdm(test_dataloader)):
  img, target = to_device(img.float(), target)
  preds = model(img)
  preds = preds > 0.5

  batch_index = i * batch_size
  submit.iloc[batch_index:batch_index+batch_size, 1:] = preds.long().squeeze(0).detach().cpu().numpy()

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 50000/50000 [11:40<00:00, 71.42it/s]


In [82]:
submit

,index,a,b,c,d,e,f,g,h,i,...,q,r,s,t,u,v,w,x,y,z
0,50000,1,1,0,1,0,0,0,1,0,...,0,1,1,0,1,0,0,1,1,1
1,50001,1,0,0,1,0,1,0,1,0,...,0,1,0,1,0,0,0,0,0,1
2,50002,0,0,0,0,0,0,0,0,1,...,0,0,1,1,1,0,0,1,1,0
3,50003,1,0,1,1,0,0,1,1,0,...,1,0,1,1,1,1,1,0,1,0
4,50004,0,0,0,1,0,1,0,0,0,...,0,1,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,54995,1,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,1,0,1,0
4996,54996,0,1,1,1,1,0,0,1,0,...,0,0,1,0,0,0,1,0,1,0
4997,54997,0,1,0,1,1,0,0,0,1,...,1,1,1,0,0,0,0,0,1,0
4998,54998,1,1,0,1,1,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1


In [83]:
submit.to_csv('submit.csv', index=False)

In [ ]:
ㅇ